In [2]:
import cv2
import numpy as np
import json
import os
from ultralytics import YOLO

In [6]:
 

class VideoRegionTracker:
    def __init__(self):
        self.points = []
        self.reference_frame = None
        self.reference_keypoints = None
        self.reference_descriptors = None
        self.window_name = "Mark Region - Click 8 Points"
        self.points_file = "tracked_region_points.json"
        
        # Initialize ORB detector for feature matching
        self.orb = cv2.ORB_create(nfeatures=500)  # Reduced features for speed
        
        # Initialize FLANN matcher
        FLANN_INDEX_LSH = 6
        index_params = dict(algorithm=FLANN_INDEX_LSH,
                           table_number=6,
                           key_size=12,
                           multi_probe_level=1)
        search_params = dict(checks=25)  # Reduced checks for speed
        self.flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN and len(self.points) < 8:
            self.points.append([x, y])
            
            # Create a copy for display
            display_img = self.reference_frame.copy()
            
            # Draw all points
            for i, point in enumerate(self.points):
                cv2.circle(display_img, tuple(point), 8, (0, 0, 255), -1)
                cv2.putText(display_img, f"{i+1}", (point[0]+15, point[1]-15), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            
            # Draw connecting lines
            if len(self.points) > 1:
                for i in range(len(self.points)-1):
                    cv2.line(display_img, tuple(self.points[i]), tuple(self.points[i+1]), (0, 255, 0), 2)
            
            # Close polygon when 8 points are marked
            if len(self.points) == 8:
                cv2.line(display_img, tuple(self.points[-1]), tuple(self.points[0]), (0, 255, 0), 2)
                cv2.putText(display_img, "Press 'q' to save", (50, 50),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            cv2.imshow(self.window_name, display_img)
    
    def extract_reference_frame(self, video_path, frame_number=10):
        """Extract reference frame and compute features"""
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print("❌ Error: Cannot open video file")
            return None
        
        # Go to specified frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
        ret, frame = cap.read()
        cap.release()
        
        if not ret:
            print(f"❌ Error: Cannot read frame {frame_number}")
            return None
        
        # Store reference frame
        self.reference_frame = frame.copy()
        
        # Convert to grayscale for feature detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect keypoints and descriptors
        self.reference_keypoints, self.reference_descriptors = self.orb.detectAndCompute(gray, None)
        
        if self.reference_descriptors is None:
            print("❌ Error: No features detected in reference frame")
            return None
        
        print(f"✅ Reference frame extracted with {len(self.reference_keypoints)} features")
        return frame
    
    def mark_region(self, video_path, frame_number=10):
        """Mark region on reference frame"""
        frame = self.extract_reference_frame(video_path, frame_number)
        if frame is None:
            return None
        
        self.points = []
        
        cv2.namedWindow(self.window_name)
        cv2.setMouseCallback(self.window_name, self.mouse_callback)
        
        print("🎯 Instructions:")
        print("   1. Click 8 points to define the bus lane region")
        print("   2. This region will be tracked throughout the video")
        print("   3. Press 'q' when done, 'r' to reset")
        
        cv2.imshow(self.window_name, frame)
        
        while True:
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q') and len(self.points) == 8:
                break
            elif key == ord('r'):
                self.points = []
                cv2.imshow(self.window_name, frame)
                print("🔄 Points reset")
            elif key == 27:  # ESC
                cv2.destroyAllWindows()
                return None
        
        cv2.destroyAllWindows()
        
        # Save points and reference frame info
        data = {
            "points": self.points,
            "reference_frame": frame_number
        }
        
        try:
            with open(self.points_file, 'w') as f:
                json.dump(data, f)
            
            print(f"✅ Bus lane region marked with {len(self.points)} points")
            return self.points
        except Exception as e:
            print(f"❌ Error saving: {e}")
            return None
    
    def find_homography(self, current_frame):
        """Find homography between reference frame and current frame"""
        # Resize for faster processing
        small_current = cv2.resize(current_frame, (640, 360))
        small_ref = cv2.resize(self.reference_frame, (640, 360))
        
        gray_current = cv2.cvtColor(small_current, cv2.COLOR_BGR2GRAY)
        gray_ref = cv2.cvtColor(small_ref, cv2.COLOR_BGR2GRAY)
        
        # Detect features in current frame
        keypoints_current, descriptors_current = self.orb.detectAndCompute(gray_current, None)
        keypoints_ref, descriptors_ref = self.orb.detectAndCompute(gray_ref, None)
        
        if descriptors_current is None or descriptors_ref is None or len(descriptors_current) < 10:
            return None
        
        try:
            # Match features
            matches = self.flann.knnMatch(descriptors_ref, descriptors_current, k=2)
            
            # Filter good matches
            good_matches = []
            for match_pair in matches:
                if len(match_pair) == 2:
                    m, n = match_pair
                    if m.distance < 0.75 * n.distance:  # Slightly relaxed for speed
                        good_matches.append(m)
            
            if len(good_matches) < 8:  # Reduced requirement
                return None
            
            # Extract matched points and scale back up
            src_pts = np.float32([keypoints_ref[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([keypoints_current[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            
            # Scale points back to original resolution
            scale_x = current_frame.shape[1] / 640
            scale_y = current_frame.shape[0] / 360
            src_pts[:, :, 0] *= scale_x
            src_pts[:, :, 1] *= scale_y
            dst_pts[:, :, 0] *= scale_x
            dst_pts[:, :, 1] *= scale_y
            
            # Find homography
            homography, mask = cv2.findHomography(src_pts, dst_pts, 
                                                cv2.RANSAC, 8.0)  # Relaxed threshold
            
            return homography
            
        except Exception as e:
            return None
    
    def transform_points(self, points, homography):
        """Transform points using homography"""
        if homography is None:
            return points
        
        # Convert points to the right format
        pts = np.float32(points).reshape(-1, 1, 2)
        
        # Transform points
        transformed_pts = cv2.perspectiveTransform(pts, homography)
        
        # Convert back to list format
        return transformed_pts.reshape(-1, 2).astype(int).tolist()
    
    def draw_region(self, frame, points, color=(0, 255, 0), thickness=2):
        """Draw the tracked region - simplified for speed"""
        if len(points) < 3:
            return
        
        pts = np.array(points, np.int32)
        pts = pts.reshape((-1, 1, 2))
        
        # Draw border only (no fill for speed)
        cv2.polylines(frame, [pts], True, color, thickness)
        
        # Draw corner points
        for point in points:
            cv2.circle(frame, tuple(point), 3, color, -1)

class EnhancedBusLaneMonitor(VideoRegionTracker):
    def __init__(self, model_path="yolov8n.pt"):
        super().__init__()
        # Load YOLO model
        try:
            self.model = YOLO(model_path)
            self.model.overrides['verbose'] = False
            print(f"✅ YOLO model loaded: {model_path}")
        except Exception as e:
            print(f"❌ Error loading YOLO model: {e}")
            self.model = None
        
        # Vehicle tracking statistics
        self.violation_count = 0
        self.authorized_count = 0
        self.total_detections = 0
        self.frame_violations = []
        
    def point_in_polygon(self, point, polygon):
        """Check if point is inside polygon using ray casting"""
        x, y = point
        n = len(polygon)
        inside = False
        
        j = n - 1
        for i in range(n):
            if ((polygon[i][1] > y) != (polygon[j][1] > y)) and \
               (x < (polygon[j][0] - polygon[i][0]) * (y - polygon[i][1]) / (polygon[j][1] - polygon[i][1]) + polygon[i][0]):
                inside = not inside
            j = i
        return inside
    
    def detect_stopped_vehicles(self, frame, region_points):
        """Special detection for stopped autorickshaws and bikes using person detection"""
        if self.model is None:
            return []
        
        # Get region mask for cropping
        mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        pts = np.array(region_points, np.int32).reshape((-1, 1, 2))
        cv2.fillPoly(mask, [pts], 255)
        
        # Get bounding box of region for cropping
        x, y, w, h = cv2.boundingRect(pts)
        
        # Add padding
        pad = 20
        x = max(0, x - pad)
        y = max(0, y - pad)
        w = min(frame.shape[1] - x, w + 2*pad)
        h = min(frame.shape[0] - y, h + 2*pad)
        
        # Crop frame to region
        cropped_frame = frame[y:y+h, x:x+w]
        cropped_mask = mask[y:y+h, x:x+w]
        
        # Run detection on cropped frame with very low confidence
        results = self.model(cropped_frame, 
                           conf=0.15,  # Very low confidence
                           iou=0.3,
                           classes=[0, 1, 2, 3, 5, 7, 9, 14],  # person, bicycle, car, motorcycle, bus, truck, boat, skateboard
                           verbose=False)
        
        detections = []
        
        for r in results:
            if r.boxes is not None:
                for box in r.boxes:
                    cls = int(box.cls[0])
                    label = r.names[cls]
                    confidence = float(box.conf[0])
                    
                    # Convert coordinates back to full frame
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    x1, y1, x2, y2 = x1 + x, y1 + y, x2 + x, y2 + y
                    
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    
                    # Check if detection is in region
                    if self.point_in_polygon((center_x, center_y), region_points):
                        
                        width = x2 - x1
                        height = y2 - y1
                        aspect_ratio = width / height if height > 0 else 0
                        area = width * height
                        
                        detected_vehicle = None
                        
                        # Enhanced detection logic
                        if label == "person":
                            # Person standing near/on vehicle - likely autorickshaw driver or bike rider
                            if height > 80:  # Tall person detection
                                # Look for vehicle-like shapes around the person
                                detected_vehicle = "autorickshaw"
                                confidence = min(confidence * 1.2, 0.9)
                        
                        elif label == "bicycle" or label == "motorcycle":
                            detected_vehicle = "motorcycle"
                            
                        elif label == "car":
                            # Many autorickshaws detected as cars
                            if 1.0 < aspect_ratio < 2.2 and 3000 < area < 12000:
                                detected_vehicle = "autorickshaw"
                                confidence = confidence * 0.9
                            else:
                                detected_vehicle = "car"
                        
                        elif label == "truck":
                            if aspect_ratio > 2.0 and area > 15000:
                                detected_vehicle = "bus"
                            elif 1.2 < aspect_ratio < 2.0 and area < 15000:
                                detected_vehicle = "autorickshaw"
                            else:
                                detected_vehicle = "truck"
                        
                        elif label == "bus":
                            detected_vehicle = "bus"
                        
                        elif label == "boat":  # Sometimes autorickshaws detected as boats
                            detected_vehicle = "autorickshaw"
                            confidence = confidence * 0.7
                        
                        if detected_vehicle:
                            self.total_detections += 1
                            
                            # Authorization logic
                            if detected_vehicle == "bus":
                                status = "AUTHORIZED"
                                color = (0, 255, 0)  # Green
                                self.authorized_count += 1
                            else:
                                status = "UNAUTHORIZED"
                                color = (0, 0, 255)  # Red
                                self.violation_count += 1
                            
                            detections.append({
                                'bbox': (x1, y1, x2, y2),
                                'center': (center_x, center_y),
                                'label': detected_vehicle,
                                'confidence': confidence,
                                'status': status,
                                'color': color,
                                'original_label': label
                            })
        
        return detections
    
    def draw_detections(self, frame, detections):
        """Draw vehicle detections - simplified for speed"""
        for detection in detections:
            x1, y1, x2, y2 = detection['bbox']
            label = detection['label']
            confidence = detection['confidence']
            status = detection['status']
            color = detection['color']
            
            # Draw bounding box
            thickness = 3 if status == "UNAUTHORIZED" else 2
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
            
            # Simple status text
            status_text = f"{status}"
            vehicle_text = f"{label.upper()}"
            
            # Draw status
            cv2.putText(frame, status_text, (x1, y1 - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
          
    
    def draw_simple_stats(self, frame, detections, frame_num):
        """Draw simplified statistics for speed"""
        violations = sum(1 for d in detections if d['status'] == "UNAUTHORIZED")
        authorized = sum(1 for d in detections if d['status'] == "AUTHORIZED")
        
        # Simple stats overlay
        stats_text = [
            f"Frame: {frame_num}",
            f"Violations: {violations}",
            f"Authorized: {authorized}",
            f"Total V: {self.violation_count}"
        ]
        
        for i, text in enumerate(stats_text):
            color = (0, 0, 255) if "Violations" in text else (255, 255, 255)
            cv2.putText(frame, text, (10, 30 + i*25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
    
    def play_video_with_bus_detection(self, video_path):
        """Play video at normal speed with optimized detection"""
        # Load saved region data
        if not os.path.exists(self.points_file):
            print("❌ No marked region found. Please mark region first.")
            return
        
        try:
            with open(self.points_file, 'r') as f:
                data = json.load(f)
            original_points = data["points"]
            reference_frame_num = data["reference_frame"]
        except Exception as e:
            print(f"❌ Error loading data: {e}")
            return
        
        # Re-extract reference frame for tracking
        if self.extract_reference_frame(video_path, reference_frame_num) is None:
            return
        
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print("❌ Error: Cannot open video file")
            return
        
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        
        # Reset counters
        self.violation_count = 0
        self.authorized_count = 0
        self.total_detections = 0
        
        print(f"🚌 Fast Bus Lane Monitoring Started")
        print(f"   📊 {total_frames} frames at {fps} FPS")
        print("   🎯 Enhanced detection for stopped vehicles")
        print("   ⌨️  Press 'q' to quit, 'p' to pause")
        
        paused = False
        frame_count = 0
        current_region_points = original_points
        
        # Initialize tracking variables
        region_color = (0, 255, 0)
        
        while True:
            if not paused:
                ret, frame = cap.read()
                frame_count += 1
                
                if not ret:
                    print("📽️ Video finished")
                    break
                
                current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                
                # Simplified tracking - update every 10 frames for speed
                if frame_count % 10 == 0:
                    homography = self.find_homography(frame)
                    if homography is not None:
                        transformed_points = self.transform_points(original_points, homography)
                        if len(transformed_points) == len(original_points):
                            current_region_points = transformed_points
                            region_color = (0, 255, 0)  # Green
                        else:
                            region_color = (0, 165, 255)  # Orange
                    else:
                        region_color = (0, 165, 255)  # Orange
                
                # Draw bus lane region (simplified)
                self.draw_region(frame, current_region_points, region_color, 2)
                
                # Add bus lane label
                cv2.putText(frame, "BUS LANE", 
                           (current_region_points[0][0], current_region_points[0][1] - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, region_color, 2)
                
                # Detect vehicles (every frame for better detection)
                detections = self.detect_stopped_vehicles(frame, current_region_points)
                
                # Draw detections
                self.draw_detections(frame, detections)
                
                # Draw simple statistics
                self.draw_simple_stats(frame, detections, current_frame_num)
            
            cv2.imshow("🚌 Fast Bus Lane Monitor", frame)
            
            # Normal video speed - calculate appropriate delay
            delay = max(1, int(1000 / fps) - 5)  # Slightly faster than real-time
            key = cv2.waitKey(delay) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('p'):
                paused = not paused
                print("⏸️ Paused" if paused else "▶️ Resumed")
        
        # Final statistics
        print(f"\n📊 Final Statistics:")
        print(f"   🚗 Total Detections: {self.total_detections}")
        print(f"   ✅ Authorized: {self.authorized_count}")
        print(f"   ❌ Violations: {self.violation_count}")
        
        cap.release()
        cv2.destroyAllWindows()

def main():
    # Initialize the monitor
    monitor = EnhancedBusLaneMonitor("yolov8n.pt")
    video_path = "testingfix.mp4"
    
    print("🚌 Fast Bus Lane Monitoring System")
    print("=" * 50)
    print("Features:")
    print("  ✅ Normal playback speed")
    print("  ✅ Detects stopped autorickshaws via person detection")
    print("  ✅ Enhanced bike/motorcycle detection")
    print("  ✅ Optimized processing")
    
    while True:
        print("\n📋 Choose an option:")
        print("1. Mark bus lane region")
        print("2. Run fast monitoring")
        print("3. View statistics")
        print("4. Exit")
        
        choice = input("Enter choice (1-4): ").strip()
        
        if choice == "1":
            frame_num = input("Frame number (default: 10): ").strip()
            try:
                frame_num = int(frame_num) if frame_num else 10
            except:
                frame_num = 10
            
            points = monitor.mark_region(video_path, frame_num)
            if points:
                print("✅ Region marked!")
                
        elif choice == "2":
            print("\n🚌 Starting fast monitoring...")
            monitor.play_video_with_bus_detection(video_path)
            
        elif choice == "3":
            print(f"\n📊 Statistics:")
            print(f"   Detections: {monitor.total_detections}")
            print(f"   Violations: {monitor.violation_count}")
                
        elif choice == "4":
            print("👋 Goodbye!")
            break
            
        else:
            print("❌ Invalid choice")

if __name__ == "__main__":
    main()

✅ YOLO model loaded: yolov8n.pt
🚌 Fast Bus Lane Monitoring System
Features:
  ✅ Normal playback speed
  ✅ Detects stopped autorickshaws via person detection
  ✅ Enhanced bike/motorcycle detection
  ✅ Optimized processing

📋 Choose an option:
1. Mark bus lane region
2. Run fast monitoring
3. View statistics
4. Exit


Enter choice (1-4):  1
Frame number (default: 10):  25


✅ Reference frame extracted with 500 features
🎯 Instructions:
   1. Click 8 points to define the bus lane region
   2. This region will be tracked throughout the video
   3. Press 'q' when done, 'r' to reset
✅ Bus lane region marked with 8 points
✅ Region marked!

📋 Choose an option:
1. Mark bus lane region
2. Run fast monitoring
3. View statistics
4. Exit


Enter choice (1-4):  2



🚌 Starting fast monitoring...
✅ Reference frame extracted with 500 features
🚌 Fast Bus Lane Monitoring Started
   📊 1410 frames at 30 FPS
   🎯 Enhanced detection for stopped vehicles
   ⌨️  Press 'q' to quit, 'p' to pause

📊 Final Statistics:
   🚗 Total Detections: 172
   ✅ Authorized: 11
   ❌ Violations: 161

📋 Choose an option:
1. Mark bus lane region
2. Run fast monitoring
3. View statistics
4. Exit


Enter choice (1-4):  4


👋 Goodbye!


In [ ]:
4